In [84]:
% matplotlib inline 

import h5py, sys, io, os, numpy as np
import nibabel as nib
import nibabel.gifti
import nilearn
from nilearn import datasets, plotting
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
mpl.rcParams['svg.fonttype'] = 'none'

UsageError: Line magic function `%` not found.


In [140]:
import pandas as pd


sys.path.append('/home/romy.beaute/projects/hypnomed/data')
from get_infos import get_rs_condition

df_path = '/home/romy.beaute/projects/hypnomed/data/hypnomed.csv'
df = pd.read_csv(df_path,sep=';',index_col='sub_id')

#check if randomisation hypnose/meditation well balanced
print(len(df[df['rs_run-2']=='hypnose'])==len(df[df['rs_run-2']=='meditation']))
print(len(df[df['rs_run-3']=='hypnose'])==len(df[df['rs_run-3']=='meditation']))

mri_infos = pd.read_csv('/home/romy.beaute/projects/hypnomed/data/MRI_database_extended.csv',sep=';')
print(mri_infos[mri_infos['check_FD (>0.15)'].notna()].sub_code) #get rid of these 2 subjects


outliers = ['sub-27','sub-32'] #list of subjects we will exclude (sub-31 does only have 2 rs)
print([df.loc[sub]['rs_run-2'] for sub in outliers]) #les deux conditions qui sont enlevées 

True
True
26    sub-27
39    sub-40
Name: sub_code, dtype: object
['hypnose', 'meditation']


- cortex = loadmat('/mnt/data/romy/hypnomed/git/data/cortex.mat')
- template_L = surface.load_surf_data(os.path.join(template_path,'hcp.embed.grad_'+str(grad_id+1)+'.L.fsa5.func.gii'))
- template_R = surface.load_surf_data(os.path.join(template_path,'hcp.embed.grad_'+str(grad_id+1)+'.R.fsa5.func.gii'))

In [177]:
from helpers_gradient import * #,load_template

vol2surf_path = '/mnt/data/romy/hypnomed/git/diffusion_embedding/vol2surf_derivatives' 
# os.listdir(vol2surf_path)

df_sublist = pd.read_csv('subject_list_updated.txt', header=None) #list of the subjects we have
sublist = np.asarray(df_sublist).flatten()
len(sublist)

emb_npys = '/mnt/data/romy/hypnomed/git/diffusion_embedding/emb_outputs'
npy_blocks = os.path.join(emb_npys,'emb_output_blocks')
npy_states = os.path.join(emb_npys,'emb_output_states')


template = load_template('/mnt/data/romy/hypnomed/git/data/template')
print(template.shape) #template of the cortex (from hcp)

(18715, 5)


In [188]:
emb_states = []
emb_blocks = []
subs = []
for sub in sublist:
    try:
        subs.append(sub)

        #for states
        emb_states.append(np.load(npy_states+f'/embedding_dense_emb.{sub}.ses-001.control.npy'))
        emb_states.append(np.load(npy_states+f'/embedding_dense_emb.{sub}.ses-001.meditation.npy'))
        emb_states.append(np.load(npy_states+f'/embedding_dense_emb.{sub}.ses-001.hypnose.npy'))

        #for blocks
        emb_blocks.append(np.load(npy_blocks+f'/embedding_dense_emb.{sub}.ses-001.rs_run-1.npy'))
        emb_blocks.append(np.load(npy_blocks+f'/embedding_dense_emb.{sub}.ses-001.rs_run-2.npy'))
        emb_blocks.append(np.load(npy_blocks+f'/embedding_dense_emb.{sub}.ses-001.rs_run-3.npy'))
        #print(sub)
    except:
        print(sub)

realigned_states = run_realign(emb_states, template)
realigned_blocks = run_realign(emb_blocks, template)
for i in range(5):
    realigned_states = run_realign(realigned_states, np.asarray(np.mean(realigned_states, axis=0).squeeze()))
    realigned_blocks = run_realign(realigned_blocks, np.asarray(np.mean(realigned_blocks, axis=0).squeeze()))

emb_folder = '/home/romy.beaute/projects/hypnomed/META/embeddings'

savemat(emb_folder+'/blocks_embeddings.mat', mdict={'emb': realigned_blocks, 'subs': subs})
savemat(emb_folder+'/states_embeddings.mat', mdict={'emb': realigned_states, 'subs': subs})
    
fb = loadmat(emb_folder+'/blocks_embeddings.mat', mdict={'emb': realigned_blocks, 'subs': subs})
fs = loadmat(emb_folder+'/states_embeddings.mat', mdict={'emb': realigned_states, 'subs': subs})

print('Group matrix (Blocks) succeded and saved in {}'.format(emb_folder+'/blocks_embeddings.mat'))
print('Group matrix (States) succeded and saved in {}'.format(emb_folder+'/states_embeddings.mat'))

Group matrix (Blocks) succeded and saved in /home/romy.beaute/projects/hypnomed/META/embeddings/blocks_embeddings.mat
Group matrix (States) succeded and saved in /home/romy.beaute/projects/hypnomed/META/embeddings/states_embeddings.mat


In [158]:
from scipy.io import loadmat
import os

#/home/romy.beaute/projects/hypnomed/diffusion_embedding/emb_matrices/emb_matrices.py


mat_folder = '/home/romy.beaute/projects/hypnomed/diffusion_embedding/emb_matrices'
mat_file = 'group_control_meditation_hypnose_embedding.mat'
b_all = loadmat(os.path.join(mat_folder,mat_file))
print(b_all['subs'])
print(b_all['states'])

emb_all = b_all['emb']
print(emb_all.shape) #(n_subjects, n_nodes, n_dims)

['sub-01' 'sub-02' 'sub-03' 'sub-04' 'sub-05' 'sub-06' 'sub-07' 'sub-08'
 'sub-09' 'sub-10' 'sub-11' 'sub-12' 'sub-13' 'sub-14' 'sub-15' 'sub-16'
 'sub-17' 'sub-18' 'sub-19' 'sub-20' 'sub-21' 'sub-22' 'sub-23' 'sub-24'
 'sub-25' 'sub-26' 'sub-27' 'sub-28' 'sub-29' 'sub-30' 'sub-31' 'sub-32'
 'sub-33' 'sub-34' 'sub-35' 'sub-36' 'sub-37' 'sub-38' 'sub-39' 'sub-40']
['control   ' 'meditation' 'hypnose   ']
(119, 18715, 5)


In [37]:
emb_all.shape

(119, 18715, 5)

In [54]:
mat_folder_group = '/home/romy.beaute/projects/hypnomed/diffusion_embedding/emb_matrices/group'
state = 'meditation'
mat_state_file = 'group_{}_embedding.mat'.format(state)
b_state = loadmat(os.path.join(mat_folder_group,mat_state_file))

print(b_state['subs'])
print(b_state['states'])

emb_state = b_state['emb']
print(emb_state.shape) #(n_subjects, n_nodes, n_dims)

pg_emb = emb_state[:,:,0] #principal (first gradient state)
print(pg_emb.shape) #(n_subjects,n_nodes)

['sub-01' 'sub-02' 'sub-03' 'sub-04' 'sub-05' 'sub-06' 'sub-07' 'sub-08'
 'sub-09' 'sub-10' 'sub-11' 'sub-12' 'sub-13' 'sub-14' 'sub-15' 'sub-16'
 'sub-17' 'sub-18' 'sub-19' 'sub-20' 'sub-21' 'sub-22' 'sub-23' 'sub-24'
 'sub-25' 'sub-26' 'sub-27' 'sub-28' 'sub-29' 'sub-30' 'sub-31' 'sub-32'
 'sub-33' 'sub-34' 'sub-35' 'sub-36' 'sub-37' 'sub-38' 'sub-39' 'sub-40']
['meditation']
(40, 18715, 5)
(40, 18715)


In [73]:
mat_folder_group = '/home/romy.beaute/projects/hypnomed/diffusion_embedding/emb_matrices/group'
state = 'meditation'
mat_state_file = 'group_{}_embedding.mat'.format(state)

b_cont = loadmat(os.path.join(mat_folder_group,'group_control_embedding.mat'))
b_med = loadmat(os.path.join(mat_folder_group,'group_meditation_embedding.mat'))
b_hyp = loadmat(os.path.join(mat_folder_group,'group_hypnose_embedding.mat'))


b_cont['subs']
b['subs'].shape
# print(b_state['subs'])
# print(b_state['states'])

# emb_state = b_state['emb']
# print(emb_state.shape) #(n_subjects, n_nodes, n_dims)

# pg_emb = emb_state[:,:,0] #principal (first gradient state)
# print(pg_emb.shape) #(n_subjects,n_nodes)

(40,)

In [60]:
import numpy as np
np.mean(pg_emb,axis=0).shape #mean across subject for each voxel

(18715,)

In [30]:
b_state['emb'][0]

array([[ 2.19480879,  0.04865987,  0.276223  , -0.87861911,  0.64333214],
       [ 1.16835411, -0.91160716, -1.81265838,  0.43102496,  0.72486073],
       [ 0.16437718, -1.25434968, -0.512887  ,  0.53932573,  1.12223924],
       ...,
       [-0.78562915, -1.91620338,  0.13634981,  1.51188669,  1.29861025],
       [-0.54179474, -1.84899268,  0.0621494 ,  1.82990362,  1.2103959 ],
       [-0.24730502, -1.75434813, -0.06249908,  2.18054246,  1.16738372]])